In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/train_waymo.yaml"   # train_tdt4265.yaml  train_waymo.yaml

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-10 11:01:47,766 SSD INFO: Loaded configuration file configs/train_waymo.yaml
2020-04-10 11:01:47,767 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet'   #vgg   #'basic'
        PRETRAINED: True
        OUT_CHANNELS: [512,1024,2048,512, 256, 128]     #[128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
        DEPTH: 50
INPUT:
    IMAGE_SIZE: 300
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
SOLVER:
    CHOICE: 'SGD'
    MAX_ITER:  65000
    LR_STEPS: [30000, 60000]
    GAMMA: 0.1
    BATCH_SIZE: 32
    LR: 1.e-2
OUTPUT_DIR: 'outputs/res50_new_waymo_sgd_1e_f2'  
DATASET_DIR: "datasets"
2020-04-10 11:01:47,767 SSD INFO: Running with config:
DATASETS:
  TEST: ('waymo_val',)
  TRAIN: ('waymo_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 8
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: 300
  PIXEL_MEAN: [123, 117, 104]
LOG_STEP: 10
MODEL:
  BACKBONE:
    AFTER_TRAINED: False
    DEPTH: 50
    INPUT_CHANNELS: 3
    N

In [ ]:
model = start_train(cfg)

resnet
Detector initialized. Total Number of params:  39.55M
Backbone number of parameters: 37.52M
SSD Head number of parameters: 2.03M
2020-04-10 11:01:51,381 SSD.trainer INFO: No checkpoint found.
17936
Dataset loaded. Subset: train, number of images: 14348
2020-04-10 11:01:52,392 SSD.trainer INFO: Start training ...
2020-04-10 11:02:04,255 SSD.trainer INFO: iter: 000010, lr: 0.00347, total_loss: 17.347 (17.347), reg_loss: 7.059 (7.059), cls_loss: 10.288 (10.288), time: 1.087 (1.087), eta: 19:36:54, mem: 9881M
2020-04-10 11:02:06,942 SSD.trainer INFO: iter: 000020, lr: 0.00360, total_loss: 12.182 (14.764), reg_loss: 6.006 (6.532), cls_loss: 6.176 (8.232), time: 0.269 (0.678), eta: 12:13:55, mem: 9881M
2020-04-10 11:02:09,919 SSD.trainer INFO: iter: 000030, lr: 0.00373, total_loss: 9.657 (13.062), reg_loss: 5.492 (6.185), cls_loss: 4.166 (6.877), time: 0.298 (0.551), eta: 9:56:38, mem: 9881M
2020-04-10 11:02:13,978 SSD.trainer INFO: iter: 000040, lr: 0.00387, total_loss: 8.645 (11.958

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)